# 1. Import Libraries

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler



# 2. Load and Combine Datasets

In [2]:
# File paths
files = {
    "hz": "delivery_hz.csv",
    "sh": "delivery_sh.csv",
    "cq": "delivery_cq.csv",
    "yt": "delivery_yt.csv",
    "jl": "delivery_delivery_jl.csv"
}

In [3]:
# Load datasets into DataFrames
dataframes = [pd.read_csv(path) for path in files.values()]

In [4]:
# Combine all datasets into one
data = pd.concat(dataframes, ignore_index=True)

In [5]:
# Display the first few rows
data.head()

,order_id,region_id,city,courier_id,lng,lat,aoi_id,aoi_type,accept_time,accept_gps_time,accept_gps_lng,accept_gps_lat,delivery_time,delivery_gps_time,delivery_gps_lng,delivery_gps_lat,ds
0,583722,0,Hangzhou,175,120.17895,30.26401,749,1,10-30 09:20:00,10-30 09:20:00,120.20600,30.28657,10-30 10:30:00,10-30 10:30:00,120.17908,30.26392,1030
1,2819059,0,Hangzhou,175,120.17899,30.26336,749,1,10-31 09:47:00,10-31 09:47:00,120.20591,30.28651,10-31 10:40:00,10-31 10:40:00,120.17884,30.26360,1031
2,2879432,0,Hangzhou,175,120.17896,30.26404,749,1,10-22 10:11:00,10-22 10:11:00,120.20598,30.28668,10-22 15:03:00,10-22 15:03:00,120.17939,30.26395,1022
3,392295,0,Hangzhou,175,120.17897,30.26408,749,1,10-26 09:41:00,10-26 09:41:00,120.20600,30.28657,10-26 10:30:00,10-26 10:30:00,120.17925,30.26465,1026
4,231864,0,Hangzhou,175,120.17888,30.26406,749,1,10-31 15:58:00,10-31 15:58:00,120.20605,30.28666,10-31 16:41:00,10-31 16:41:00,120.17886,30.26402,1031


In [6]:
# Save the merged dataset
data.to_csv("merged_delivery_data.csv", index=False)
print("Merged data saved to 'merged_delivery_data.csv'")

Merged data saved to 'merged_delivery_data.csv'


In [7]:
# Load the dataset
df = pd.read_csv('merged_delivery_data.csv')

In [8]:
# Display the first few rows
data.head()

,order_id,region_id,city,courier_id,lng,lat,aoi_id,aoi_type,accept_time,accept_gps_time,accept_gps_lng,accept_gps_lat,delivery_time,delivery_gps_time,delivery_gps_lng,delivery_gps_lat,ds
0,583722,0,Hangzhou,175,120.17895,30.26401,749,1,10-30 09:20:00,10-30 09:20:00,120.20600,30.28657,10-30 10:30:00,10-30 10:30:00,120.17908,30.26392,1030
1,2819059,0,Hangzhou,175,120.17899,30.26336,749,1,10-31 09:47:00,10-31 09:47:00,120.20591,30.28651,10-31 10:40:00,10-31 10:40:00,120.17884,30.26360,1031
2,2879432,0,Hangzhou,175,120.17896,30.26404,749,1,10-22 10:11:00,10-22 10:11:00,120.20598,30.28668,10-22 15:03:00,10-22 15:03:00,120.17939,30.26395,1022
3,392295,0,Hangzhou,175,120.17897,30.26408,749,1,10-26 09:41:00,10-26 09:41:00,120.20600,30.28657,10-26 10:30:00,10-26 10:30:00,120.17925,30.26465,1026
4,231864,0,Hangzhou,175,120.17888,30.26406,749,1,10-31 15:58:00,10-31 15:58:00,120.20605,30.28666,10-31 16:41:00,10-31 16:41:00,120.17886,30.26402,1031


# 1. Data Cleaning

In [9]:
# Handle missing values
critical_columns = ['accept_time', 'delivery_time', 'lng', 'lat']
for column in critical_columns:
    if df[column].dtype in ['float64', 'int64']:
        df[column].fillna(df[column].mean(), inplace=True)  # Fill numerical columns with mean
    else:
        df[column].fillna('Unknown', inplace=True)  # Fill categorical columns with 'Unknown'

In [10]:
# Remove duplicate entries based on `package_id`
df.drop_duplicates(subset=['courier_id'], inplace=True)

In [11]:
# Verify and correct data types
date_columns = ['accept_time', 'delivery_time', 'accept_gps_time', 'delivery_gps_time', 'ds']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S', errors='coerce')

In [12]:
# Convert `lng` and `lat` to float if not already
df['lng'] = df['lng'].astype(float, errors='ignore')
df['lat'] = df['lat'].astype(float, errors='ignore')


# 2. Outlier Detection


In [13]:
# Anomalies in delivery times (e.g., delivery_time earlier than accept_time)
df = df[df['delivery_time'] >= df['accept_time']]

In [14]:
# Anomalies in geospatial coordinates (e.g., invalid lat/lng values)
df = df[(df['lng'] >= -180) & (df['lng'] <= 180)]
df = df[(df['lat'] >= -90) & (df['lat'] <= 90)]

# 3. Data Transformation

In [15]:
# Encode categorical variables
categorical_columns = ['city', 'aoi_type']
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le


In [17]:
# Normalize numerical features
from sklearn.preprocessing import MinMaxScaler

numerical_columns = ['lng', 'lat', 'accept_gps_lng', 'accept_gps_lat', 'delivery_gps_lng', 'delivery_gps_lat']

# Drop rows with missing values in numerical columns
df.dropna(subset=numerical_columns, inplace=True)

# Fill any remaining missing values with column mean
df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].mean())

# Ensure there is data before scaling
if not df[numerical_columns].empty:
    scaler = MinMaxScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
    print("Normalization completed.")
else:
    print("No data to scale in numerical columns.")


No data to scale in numerical columns.


In [18]:
# Save the cleaned and processed dataset
data.to_csv("cleaned_delivery_data.csv", index=False)
print("Cleaned data saved to 'cleaned_delivery_data.csv'")

Cleaned data saved to 'cleaned_delivery_data.csv'
